In [1]:

import scrapy, csv, sys, os
from datetime import datetime, timedelta
from scrapy.crawler import CrawlerProcess
from tqdm import tqdm

In [ ]:
class HespressSpider(scrapy.Spider):
    name = 'hespress'
    allowed_domains = ['hespress.com']
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    # user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    # user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
    def clear(self):
        # sys.stderr.flush()
        sys.stdout.flush()
        os.system('cls' if os.name=='nt' else 'clear')
        f = open('log', "w")
        f.close()

    def start_requests(self):
        with open("HespressComments.csv","w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f,fieldnames = ['title', 'type', 'link', 'date', 'comments'])
            writer.writeheader()
        
        self.bar = tqdm(1e+15, desc= "nb comment")#, file = sys.stdout)
        self.comment_date = ''
        # articles du mois septembre et octobre
        url = "https://www.hespress.com/archive/"
        date = datetime.today()
        # date_time_str = 'Jun 31 2019  12:00PM'
        # date = datetime.strptime(date_time_str, '%b %d %Y %I:%M%p')
        date_time_str = '2019-12-30 08:15:27.243860'
        date = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
         
        for _ in range(10):
            date -= timedelta(days=1)
            for i in range(1,11):
                try:
                    url = url +str(date.year)+"/"+str(date.month)+"/"+str(date.day)+ "/index."+ str(i) +".html"
                    self.comment_date = str(date.year)+"/"+str(date.month)+"/"+str(date.day)
                    self.clear()
                    yield scrapy.Request(url = url, callback = self.getUrls)
                except:
                    continue
        
    def getUrls(self, response):
        urls = response.xpath('//div[@id="box_center_holder"]//div[@class="short"]//h2[@class="section_title"]//a/@href').extract()
        # titres = response.xpath('//div[@id="box_center_holder"]//div[@class="short"]//h2[@class="section_title"]//a/text()').extract()
        # for url,titre in zip(urls,titres):
        #     yield scrapy.Request(response.urljoin(url), self.parse)
        # with open(r'D:/WISD/S3/Web_Mining/Scrapy_Spider/URLs.txt', 'a', encoding="utf-8") as f:
        #     f.write("\n".join(urls))
        for url in zip(urls):
            yield scrapy.Request(response.urljoin(str(url)), self.parse)
          
    def parse(self, response):
        items = {}
        items["title"] = response.xpath('//div[@id="article_holder"]//h1[@class="page_title"]/text()').extract()[0]
        # items["date"]  = response.xpath('//div[@id="article_body"]//div[@class="story_stamp"]//spam/text()').extract()
        # items["date"]  = response.xpath('//div[@id="comment_list"]//div[@class="comment_holder"]//div[@class="irow_1"]//div[@class="comment_body_in"]//div[@class="comment_body"]//div[@class="comment_header"]//spam/text()').extract()
        items["date"]  = self.comment_date
        items["link"]  = response.xpath('//head//link/@href').extract()[0]
        items["type"]  = items["link"].split("/")[-2]
        comments = response.xpath('//div[@id="comment_list"]//div[@class="comment_text"]/text()').extract()
                
        with open("HespressComments.csv", "a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames = ['title', 'type', 'link', 'date', 'comments'])
            for c in comments:

                items["comments"] = str(c)
                # if c:
                #     items["comments_2"] = str(tp().text_pros(c))
                # else:
                #     items["comments_2"] = ""
                writer.writerow(items)
                # self.itemlist.append(items)

                self.bar.update(1)
                # yield items
            
        # with open(r'D:/WISD/S3/Web_Mining/Scrapy_Spider/comments.txt', 'a', encoding="utf-8") as f:
        #     f.write("\n".join(comments))        

# process = CrawlerProcess({
#     'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
# })

# process.crawl(HespressSpider)
# process.start() # the script will block here until the crawling is finished